In [3]:
import pandas as pd
import numpy as np

#### Prepare Ratings Data 

The ratings dataset contains approximately 27,000,000 ratings. This may require more processing power and memory than a personal computer is able to handle. We are first going to explore the data and then decide if we should split it based on a metric or just randomly select a percentage of the data. The decision will ultimately be made by testing different sizes on our chosen Machine Learning model.

In [2]:
ratings = pd.read_csv('Data/ratings.csv')

FileNotFoundError: [Errno 2] File b'Data/ratings.csv' does not exist: b'Data/ratings.csv'

In [5]:
print(ratings.shape)
ratings.head()

(26024289, 4)


,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [23]:
# no. of unique users

user_ct = ratings['userId'].unique()

print(len(user_ct))

# of unique movies

movie_ct = ratings['movieId'].unique()

print(len(movie_ct))

270896
45115


In [9]:
movies = pd.read_csv('Data/movies_final')

movies.head()

,Unnamed: 0,movieId,genres,Title,year,Genre,Actors,Director,Plot,Poster
0,0,1,Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995,"Animation, Adventure, Comedy, Family, Fantasy","Tom Hanks, Tim Allen, Don Rickles, Jim Varney",John Lasseter,A cowboy doll is profoundly threatened and jea...,https://m.media-amazon.com/images/M/MV5BMDU2ZW...
1,1,2,Adventure|Children|Fantasy,Jumanji,1995,"Adventure, Comedy, Family, Fantasy","Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",Joe Johnston,When two kids find and play a magical board ga...,https://m.media-amazon.com/images/M/MV5BZTk2Zm...
2,2,3,Comedy|Romance,Grumpier Old Men,1995,"Comedy, Romance","Walter Matthau, Jack Lemmon, Sophia Loren, Ann...",Howard Deutch,John and Max resolve to save their beloved bai...,https://m.media-amazon.com/images/M/MV5BMjQxM2...
3,3,4,Comedy|Drama|Romance,Waiting to Exhale,1995,"Comedy, Drama, Romance","Whitney Houston, Angela Bassett, Loretta Devin...",Forest Whitaker,"Based on Terry McMillan's novel, this film fol...",https://m.media-amazon.com/images/M/MV5BYzcyMD...
4,4,5,Comedy,Father of the Bride Part II,1995,"Comedy, Family, Romance","Steve Martin, Diane Keaton, Martin Short, Kimb...",Charles Shyer,George Banks must deal not only with the pregn...,https://m.media-amazon.com/images/M/MV5BOTEyNz...


In [10]:
movies.columns


Index(['Unnamed: 0', 'movieId', 'genres', 'Title', 'year', 'Genre', 'Actors',
       'Director', 'Plot', 'Poster'],
      dtype='object')

In [11]:
movies.drop(['Unnamed: 0'], axis=1, inplace = True)

,movieId,genres,Title,year,Genre,Actors,Director,Plot,Poster
0,1,Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995,"Animation, Adventure, Comedy, Family, Fantasy","Tom Hanks, Tim Allen, Don Rickles, Jim Varney",John Lasseter,A cowboy doll is profoundly threatened and jea...,https://m.media-amazon.com/images/M/MV5BMDU2ZW...
1,2,Adventure|Children|Fantasy,Jumanji,1995,"Adventure, Comedy, Family, Fantasy","Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",Joe Johnston,When two kids find and play a magical board ga...,https://m.media-amazon.com/images/M/MV5BZTk2Zm...
2,3,Comedy|Romance,Grumpier Old Men,1995,"Comedy, Romance","Walter Matthau, Jack Lemmon, Sophia Loren, Ann...",Howard Deutch,John and Max resolve to save their beloved bai...,https://m.media-amazon.com/images/M/MV5BMjQxM2...
3,4,Comedy|Drama|Romance,Waiting to Exhale,1995,"Comedy, Drama, Romance","Whitney Houston, Angela Bassett, Loretta Devin...",Forest Whitaker,"Based on Terry McMillan's novel, this film fol...",https://m.media-amazon.com/images/M/MV5BYzcyMD...
4,5,Comedy,Father of the Bride Part II,1995,"Comedy, Family, Romance","Steve Martin, Diane Keaton, Martin Short, Kimb...",Charles Shyer,George Banks must deal not only with the pregn...,https://m.media-amazon.com/images/M/MV5BOTEyNz...


In [13]:
# combine ratings and movies. only keep ratings where the movie exists in our dataset

movie_w_ratings = pd.merge(ratings, movies,how = 'inner', on='movieId')

In [14]:
print(movie_w_ratings.shape)
movie_w_ratings.head()

(15178941, 12)


,userId,movieId,rating,timestamp,genres,Title,year,Genre,Actors,Director,Plot,Poster
0,1,110,1.0,1425941529,Action|Drama|War,Braveheart,1995,"Biography, Drama, History, War","James Robinson, Sean Lawlor, Sandy Nelson, Jam...",Mel Gibson,When his secret bride is executed for assaulti...,https://m.media-amazon.com/images/M/MV5BMzkzMm...
1,11,110,3.5,1231676989,Action|Drama|War,Braveheart,1995,"Biography, Drama, History, War","James Robinson, Sean Lawlor, Sandy Nelson, Jam...",Mel Gibson,When his secret bride is executed for assaulti...,https://m.media-amazon.com/images/M/MV5BMzkzMm...
2,22,110,5.0,1111937009,Action|Drama|War,Braveheart,1995,"Biography, Drama, History, War","James Robinson, Sean Lawlor, Sandy Nelson, Jam...",Mel Gibson,When his secret bride is executed for assaulti...,https://m.media-amazon.com/images/M/MV5BMzkzMm...
3,24,110,5.0,979870012,Action|Drama|War,Braveheart,1995,"Biography, Drama, History, War","James Robinson, Sean Lawlor, Sandy Nelson, Jam...",Mel Gibson,When his secret bride is executed for assaulti...,https://m.media-amazon.com/images/M/MV5BMzkzMm...
4,29,110,3.0,1044020005,Action|Drama|War,Braveheart,1995,"Biography, Drama, History, War","James Robinson, Sean Lawlor, Sandy Nelson, Jam...",Mel Gibson,When his secret bride is executed for assaulti...,https://m.media-amazon.com/images/M/MV5BMzkzMm...


In [22]:
# find the number of unique users and unique movies

user_count = movie_w_ratings['userId'].unique()

print(len(user_count))

movie_count = movie_w_ratings['movieId'].unique()

print(len(movie_count))

267150
22203


The number of unique movies decreased from 45,115 to 22,203. The number of unique users decreased from 270,896 to 267,150. Despite the big drop in the number of movies, we still have 15,178,941 ratings.

In [24]:
# make new dataframe with only movieId, title, rating

new_ratings = movie_w_ratings[['movieId', 'Title', 'rating']]